In [16]:
# device = 'K0303'
# active_power_variable = 'active_power'
path_base_r_string = r'D:\OneDrive - CELSIA S.A E.S.P'

%load_ext autoreload
%autoreload 2
import warnings
# warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# this cell enables project path relative imports
import sys
from pathlib import Path
path_base = Path(path_base_r_string)
project_path = path_base / 'Proyectos' / 'Eficiencia_Energetica' / 'Homecenter' / 'medicion_temporal_cargill'
sys.path.append(str(project_path))

In [18]:
# import all your modules here
import json
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

import config as cfg

from library_ubidots_v2 import Ubidots as ubi
from library_report_v2 import Cleaning as cln
from library_report_v2 import Graphing as grp
from library_report_v2 import Processing as pro


In [19]:
df = pd.read_pickle(project_path / "dataframe.pkl")
df_tag_aa_ilu = pd.read_excel("ubidots_device_map.xlsx", sheet_name='AA-ILU')

df = pd.merge(
    df.reset_index(),
    df_tag_aa_ilu,
    how='inner'
)

df = df.set_index('datetime')

df['value'] = pd.to_numeric(df['value'], errors='coerce')

In [ ]:
dates_to_remove = {
    
    'cali-norte---torre-de-enfriamiento':[
        ['2022-01-01','2022-05-25']
    ],
    'cali-sur---aires-acondicionados':[
        ['2022-01-01','2022-05-25']
    ],
    'cedritos---iluminacion-patio-constructor':[
        ['2022-01-01','2022-05-20']
    ],
    
    'cali-norte---iluminacion-parqueaderos-2':[
        ['2022-01-01','2022-04-30']
    ],
    'cali-norte---iluminacion-principal-2':[
        ['2022-01-01','2022-05-25']
    ],
    'iluminacion---primer-piso':[ # cedritos
        ['2022-01-01','2022-12-30']
    ],
    'cedritos-iluminacion-principal':[
        ['2022-05-15','2022-12-30']
    ],
    'bello---iluminacion-patio-constructor':[
        ['2022-01-01','2022-05-25']
    ],
    'tintal---iluminacion-parqueaderos':[
        ['2022-01-01','2022-06-20']
    ],
    'baq---aire-acondicionado-piso-ventas':[
        ['2022-01-01','2022-05-20']
    ],
}

energy_var_rename = {
    'ener':'energia-activa-acumulada',
    'energia-activa':'energia-activa',
    'energia-activa-acumulada':'energia-activa-acumulada',
    'energia-activa-calculada':'energia-activa'
}

In [44]:
# df_energy = df[df['variable'].isin(energy_var_rename.keys())].copy()
# df_energy['variable'] = df_energy['variable'].map(energy_var_rename)

df_cons = df.query('variable == "energia-activa"').copy()
df_cumsum = cln.recover_energy_from_consumption(df_cons, new_varname='energia-activa-acumulada')
df_ea = pd.concat([df_cons, df_cumsum])

df_cumsum_rep = cln.repair_monotonic_increasing_variable(df_cumsum, max_iter=15, tolerance=0.05, trust_gaps=False)
df_pa_synth = cln.differentiate_single_variable(df_cumsum, 'potencia-activa-sintetica', remove_gap_data=False)
df_pa_synth = cln.remove_outliers_by_zscore(df_pa_synth, zscore=5)

df_test = cln.subset_discard_date_intervals(df_pa_synth, dates_to_remove)
df_test = pro.datetime_attributes(df_test)

# remove negative values

df_test = df_test.query("value >= 0")

- Split into offices (one section per office)
- Then split into AA and ILU (one plot each)
- Plot the hourly mean for every circuit in each office


Plotly curva horaria por circuito en la sede:
- Para AA
- Para Iluminación
- Para el total

Curva de KPI (kWh/m^2/año) por sede

Series de tiempo de consumos mensuales (bar plot)

Figura de consumo energético por tipo de día de la semana



Bell y San Juan dejan la iluminación prendida


In [88]:
def office_cell_function(df_office, office_name):

    df_day_by_hour = (
        df_office
        .reset_index()
        .groupby(['office_label','type','device_name','hour'])['value']
        .agg(['median','mean','std','min',pro.q_low,pro.q_high,'max','count'])
        .reset_index()
    )

    df_week_by_day = (
        df_office
        .reset_index()
        .groupby(['office_label','type','device_name','cont_dow'])['value']
        .agg(['median','mean','std','min',pro.q_low,pro.q_high,'max','count'])
        .reset_index()
    )

    grp.plot_typical_day_by_hour(
        df_day_by_hour.query("type == 'AA'"), 
        subset='device_name', 
        title=f"Día típico para los equipos de climatización en la sede de {office_name}", 
        legend=True, 
        include_ci=True, 
        fill_ci=True
    )

    grp.plot_typical_day_by_hour(
        df_day_by_hour.query("type == 'ILU'"), 
        subset='device_name', 
        title=f"Día típico para la iluminación en la sede de {office_name}", 
        legend=True, 
        include_ci=True, 
        fill_ci=True
    )
    
    grp.plot_typical_week_by_day(
        df_week_by_day.query("type == 'AA'"),
        subset='device_name',  
        title=f"Semana típica para los equipos de climatización en la sede de {office_name}",
        legend=True, 
        include_ci=True, 
        fill_ci=True
    )

    grp.plot_typical_week_by_day(
        df_week_by_day.query("type == 'ILU'"),
        subset='device_name',  
        title=f"Semana típica para la iluminación en la sede de {office_name}",
        legend=True, 
        include_ci=True, 
        fill_ci=True
    )

In [89]:
office_name = 'Bello'
office_label = 'hc-bello'
df_office = df_test.query("office_label == @office_label")
office_cell_function(df_office, office_name)

In [85]:
# import holidays
# import datetime as dt

  
# # Select country
# co_holidays = holidays.Colombia(years = 2022)

# for date in co_holidays.keys():
#     # print(date, holiday)
#     # print(dt.datetime(date))
#     print(date.strftime('%w'))

In [ ]:
for device_name in set(df_ea['device_name']):
    df_test_dev = df_test.query("device_name == @device_name")

    plt.figure(figsize=(21,7))
    plt.plot(df_test_dev.value)
    plt.title(device_name)
    plt.show()

In [ ]:
df_plot = df_ea.query("variable == 'energia-activa-acumulada'")
for device_name in set(df_ea['device_name']):
    df_ea_device = df_plot.query("device_name == @device_name")
    df_cumsum_rep_device = df_cumsum_rep.query("device_name == @device_name")

    plt.figure(figsize=(21,7))
    plt.plot(df_ea_device.value)
    plt.plot(df_cumsum_rep_device.value)
    plt.title(device_name)
    plt.show()

In [ ]:
# df_energy_vars = df_energy.groupby(by=['device_name','variable'])['value'].count().reset_index()

# df_energy_vars['bool'] = True
# df_energy_vars_wide = (
#     df_energy_vars
#     .pivot(
#         index='variable',
#         columns='device_name',
#         values='bool'
#     )
#     .replace({np.nan:0})
# )
# # df_energy_vars_wide.transpose().sort_index()

In [ ]:
# repair_monotonic_increasing_variable(df, max_iter=None, tolerance=None, trust_gaps=False)
# Hay que irse con la energía activa y limpiarla. Aplicate un cumsum y luego el método de limpieza propio.
